In [1]:
!which python

/cluster/work/bewi/members/tuncel/anaconda3/envs/r-3.4.1-env/bin/python


In [2]:
import pandas as pd
import numpy as np
from scipy.io import mmread
import loompy

/cluster/work/bewi/members/tuncel/anaconda3/envs/r-3.4.1-env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
import h5py

In [53]:
data_path = '/cluster/work/bewi/members/tuncel/data/cellranger_run/20150306_22_27/outs/filtered_gene_bc_matrices/hg19/'
filename_data = data_path+'matrix.mtx'
filename_genes = data_path+'./genes.tsv'
filename_barcodes = data_path+'./barcodes.tsv'

In [54]:
type(filename_data)

str

In [55]:
# to read a matrix market file
matrix = mmread(filename_data).astype("float32").todense().T


In [56]:
matrix

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [57]:
matrix.shape

(2216, 32738)

In [58]:
np.asarray(matrix).shape

(2216, 32738)

In [59]:
detected_genes_index = ~(np.asarray(matrix) == 0).all(axis=0)

In [60]:
sum(detected_genes_index)

19928

In [61]:
matrix.shape

(2216, 32738)

In [52]:
matrix[:,detected_genes_index].shape

(2216, 19928)

In [19]:
gene_names = np.genfromtxt(filename_genes, dtype=str)[:, 1]

In [20]:
gene_names

array(['MIR1302-10', 'FAM138A', 'OR4F5', ..., 'CU459201.1', 'AC002321.2',
       'AC002321.1'], dtype='<U19')

In [46]:
gene_names.shape

(32738,)

In [47]:
gene_names[detected_genes_index].shape

(19928,)

In [21]:
cell_names = np.genfromtxt(filename_barcodes, dtype=str)

In [22]:
matrix.shape

(32738, 2216)

In [8]:
len(gene_names)

33694

In [9]:
len(cell_names)

2495

In [10]:
col_attrs = { "cells": cell_names }
row_attrs = { "genes": gene_names }

In [ ]:
loompy.create('./loom_files/melanoma_S2.loom', matrix=matrix, row_attrs=row_attrs, col_attrs=col_attrs)

In [13]:
!ls loom_files/

adatawritten_melanoma_S2.loom
melanoma_S2.loom
melanoma_S2_seurat_filtered.loom
splatter_seurat_filtered_melanoma_S2_group_3.loom


In [14]:
ds = loompy.connect("./loom_files/melanoma_S2.loom")

In [15]:
ds

,cells,AAACCTGAGGGCATGT-1,AAACCTGAGTACTTGC-1,AAACCTGCATCTCCCA-1,AAACCTGGTAAGGGCT-1,AAACCTGGTACCGTTA-1,AAACCTGGTCTCCATC-1,AAACCTGGTTTGGCGC-1,AAACCTGTCCGAAGAG-1,AAACGGGAGAGTCGGT-1,AAACGGGGTGCACCAC-1,...
genes,,,,,,,,,,,,...
RP11-34P13.3,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...
FAM138A,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...
OR4F5,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...
RP11-34P13.7,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...
RP11-34P13.8,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...
RP11-34P13.14,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...
RP11-34P13.9,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...
FO538757.3,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...
FO538757.2,,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...


The main matrix is availabe as a layer named “” (the empty string). It cannot be deleted but otherwise supports the same operations as any other layer.

In [20]:
len(ds.row_attrs['genes'])

33694

In [21]:
len(ds.col_attrs['cells'])

2495

In [5]:
ds.layer[''][1:5,1:5]

array([[ 11.35636675,  29.2262771 ,   0.48675933,  23.63540793],
       [  7.42552481,   2.24896409,   5.45899307,  13.19905957],
       [  8.54834756,  12.3210227 ,  15.76393424,  34.5654282 ],
       [ 24.09572588,  34.53504552,  12.3902842 ,   3.68457064]])

In [17]:
ds[:,:].shape

(33694, 2495)

In [22]:
ds.close()